In [50]:
import os, sys, time
#IMPORTANT: RUN SCRIPT FROM THIS SCRIPTS DIRECTORY i.e data_exploration/ TODO: make more robust
##os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../../scripts')
from iso_country_codes import AFRICA_CC



import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
import geoplot
import matplotlib.pyplot as plt
from iso_country_codes import AFRICA_CC
from osm_pbf_power_data_extractor import convert_pd_to_gdf_lines, convert_pd_to_gdf

import logging
logger = logging.getLogger(__name__)

In [51]:
#----------- SUBSTATIONS -----------
# Load uncleaned data
df_all_substations = pd.read_csv(os.getcwd()+"/data/africa_all_substations.csv")
# Clean
df_all_substations = df_all_substations.reset_index(drop=True)
df_all_substations = df_all_substations.dropna(subset=['tags.voltage']) # Drop any substations with Voltage = N/A

#~ Generate Files

#CSV
# outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_substations'+'_cleaned.')
# df_all_substations.to_csv(outputfile_partial + 'csv') # Generate CSV

#GeoJSON
##gdf_substations = convert_pd_to_gdf(df_all_substations)
##gdf_substations.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson


display(df_all_substations)

,Unnamed: 0,lonlat,Type,Country,tags.voltage,tags.frequency,tags.substation,tags.power,id
53,5,[ 8.5891307 31.78097517],Way,algeria,66000,NaN,industrial,substation,695874654.0
58,10,[ 6.95047259 36.71837818],Way,algeria,400000;22000;66000,NaN,transmission,substation,176932212.0
59,11,[ 0.39731576 36.09139694],Way,algeria,60000,NaN,NaN,substation,381467892.0
61,13,[ 0.128517 36.01296688],Way,algeria,220000,NaN,transmission,substation,174171686.0
64,16,[-0.16862276 35.766583 ],Way,algeria,400000;220000,NaN,transmission,substation,751040071.0
...,...,...,...,...,...,...,...,...,...
6863,123,[ 32.38763605 -20.23367165],Way,zimbabwe,132000,NaN,transmission,substation,461016797.0
6864,124,[ 32.53745103 -18.99083697],Way,zimbabwe,330000;132000,NaN,transmission,substation,211927848.0
6871,131,[ 30.51697798 -19.2954121 ],Way,zimbabwe,132000,NaN,transmission,substation,387749732.0
6875,135,[ 31.0928283 -17.85747787],Way,zimbabwe,132000,NaN,transmission,substation,406067564.0


In [52]:
    # ----------- LINES -----------

    # Clean
    # TODO: FIX Voltage Filter
    # Some transmission lines carry multiple voltages, having voltage_V = 10000;20000  (two lines)
    # The following code keeps only the first information before the semicolon..
    # Needs to be corrected in future, creating two lines with the same bus ID.

# Load unclean data
df_all_lines = pd.read_csv(os.getcwd()+"/data/africa_all_lines.csv")    

# Clean data    
df_all_lines = df_all_lines.reset_index(drop=True)
df_all_lines = df_all_lines.dropna(subset=['tags.voltage']) # Drop any lines with Voltage = N/A
df_all_lines = df_all_lines.rename(columns = {'tags.voltage':"voltage_V"}) 
df_all_lines['voltage_V'] = df_all_lines['voltage_V'].str.split('*').str[0] #just keeps the 
df_all_lines['voltage_V'] = df_all_lines['voltage_V'].str.split(';').str[0]
df_all_lines['voltage_V'] = df_all_lines['voltage_V'].apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna() ## if cell can't converted to float -> drop
df_all_lines = df_all_lines[df_all_lines.voltage_V > 10000]


## Generate Files

#CSV
# outputfile_partial = os.path.join(os.getcwd(), 'data', 'africa_all'+'_lines'+'_cleaned.')  
# df_all_lines.to_csv(outputfile_partial + 'csv')  # Generate CSV

#GeoJSON
## gdf_lines = convert_pd_to_gdf_lines(df_all_lines, simplified=True)
## gdf_lines.to_file(outputfile_partial+'geojson',
##             driver="GeoJSON")  # Generate GeoJson

display(df_all_lines)

,Unnamed: 0,tags.cables,tags.circuits,lonlat,Type,Country,voltage_V,tags.frequency,tags.power,id
0,0,6.0,2.0,"[(7.638754199999989, 36.46324789999969), (7.63...",Way,algeria,66000.0,NaN,line,686356369
4,4,3.0,NaN,"[(4.482109400000002, 35.70698170000006), (4.48...",Way,algeria,220000.0,NaN,line,218514369
5,5,3.0,NaN,"[(2.839576300000001, 36.65068540000005), (2.83...",Way,algeria,60000.0,NaN,line,685707253
6,6,3.0,NaN,"[(-0.9018965999999973, 35.087609400000055), (-...",Way,algeria,220000.0,50.0,line,380986682
7,7,3.0,NaN,"[(6.175978799999999, 35.56709889999991), (6.17...",Way,algeria,66000.0,NaN,line,226337739
...,...,...,...,...,...,...,...,...,...,...
21396,277,3.0,1.0,"[(29.77780860000017, -18.75765889999997), (29....",Way,zimbabwe,330000.0,50.0,line,194106240
21398,279,6.0,NaN,"[(30.897846299999877, -17.854492600000032), (3...",Way,zimbabwe,132000.0,NaN,line,241026076
21399,280,3.0,NaN,"[(30.956184200000163, -17.852611099999482), (3...",Way,zimbabwe,132000.0,NaN,line,241026080
21401,282,3.0,NaN,"[(30.162582300000008, -19.34563870000007), (30...",Way,zimbabwe,132000.0,NaN,line,387749730


In [43]:
# ----------- Generator -----------

#Load uncleaned data
df_all_generators = pd.read_csv(os.getcwd()+"/data/africa_all_generators.csv")

#Clean data
df_all_generators = df_all_generators.reset_index(drop=True)
df_all_generators = df_all_generators[df_all_generators['tags.generator:output:electricity'].astype(str).str.contains('MW')] #removes boolean 
df_all_generators['tags.generator:output:electricity'] = df_all_generators['tags.generator:output:electricity'].str.extract('(\d+)').astype(float)
df_all_generators = df_all_generators.rename(columns = {'tags.generator:output:electricity':"power_output_MW"})


## Generate Files

#CSV
#outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_generators'+'_cleaned.')
#df_all_generators.to_csv(outputfile_partial + 'csv') # Generate CSV

#GeoJSON
# gdf_generators = convert_pd_to_gdf(df_all_generators)
# gdf_generators.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson


display(df_all_generators)

,Unnamed: 0,tags.generator:method,power_output_MW,lonlat,Type,Country,tags.generator:source,tags.generator:type,tags.power,id
0,0,combustion,20.0,"[-0.2960970999999994, 27.893881099999952]",Node,algeria,gas,gas_turbine,generator,7.781385e+09
1,1,combustion,155.0,"[0.5403953999999923, 35.7832673999999]",Node,algeria,gas,gas_turbine,generator,5.768871e+09
2,2,combustion,140.0,"[3.1371707, 36.59923709999994]",Node,algeria,gas,gas_turbine,generator,7.781385e+09
4,4,combustion,24.0,"[3.0813003000000023, 36.749166399999964]",Node,algeria,gas,gas_turbine,generator,7.781385e+09
6,6,combustion,24.0,"[4.479375900000002, 35.70615449999996]",Node,algeria,gas,gas_turbine,generator,7.781385e+09
...,...,...,...,...,...,...,...,...,...,...
11732,18,combustion,120.0,"[26.47055979999989, -18.382236400000068]",Node,zimbabwe,coal,steam_turbine,generator,7.983191e+09
11733,19,NaN,125.0,"[28.764393099999932, -16.521852699999975]",Node,zimbabwe,hydro,NaN,generator,7.979052e+09
11734,20,combustion,120.0,"[26.470797099999892, -18.38285520000007]",Node,zimbabwe,coal,steam_turbine,generator,7.983191e+09
11735,21,NaN,180.0,"[28.762126599999934, -16.51966129999997]",Node,zimbabwe,hydro,NaN,generator,7.979052e+09
